# Isolation Project Lecture Notes
### December 2017, by Jude Moon

windows, Python3.6, [aind-environment-windows.yml](http://video.udacity-data.com.s3.amazonaws.com/topher/2017/April/58ee7f6c_aind-environment-windows/aind-environment-windows.yml)
***

## Main Topics
1. Adversarial Search
2. MiniMax Algorithm
3. Alpha-Beta Pruning
4. Evaluation Functions
5. (Application) Isolation Game Player
6. Examine multi-players & Probabilistic Games

## What is Isolation Game?
It is played by two players on a 5x5 board. A move can be made to a neighboring (horizontally, vertically, or diagonally) position that contains a current square and mark the current square as "has been" square. The player who cannot make any move loses the game.

## Objective
The objective in this course is to build a isolation game player, using AI techniques.

## Concepts:
- O: computer player move 
- X: an opponent
- +1: win by the computer player
- -1: win by the opponent
- △: turn when the computer player is trying to maximize the score
- ▽: turn thwn the opponent player is trying to minimize the score
- See the [Tree](https://s3.amazonaws.com/content.udacity-data.com/courses/ud954/images/isolation-L6_minMax_complete.png)

## MiniMax Algorithm

### Part 1: Minimax Search Pseudocode 
![image](https://d17h27t6h515a5.cloudfront.net/topher/2017/July/596d2f7f_minimax-decision/minimax-decision.png)
### Part 2: Game State Representation

In [1]:
xlim, ylim = 3, 2
_board = [[0] * ylim for _ in range(xlim)]

In [2]:
_board

[[0, 0], [0, 0], [0, 0]]

In [ ]:
from copy import deepcopy

xlim, ylim = 3, 2  # board dimensions

class GameState:
    """
    Attributes
    ----------
    _board: list(list)
        Represent the board with a 2d array _board[x][y]
        where open spaces are 0 and closed spaces are 1
    
    _parity: bool
        Keep track of active player initiative (which
        player has control to move) where 0 indicates that
        player one has initiative and 1 indicates player 2
    
    _player_locations: list(tuple)
        Keep track of the current location of each player
        on the board where position is encoded by the
        board indices of their last move, e.g., [(0, 0), (1, 0)]
        means player 1 is at (0, 0) and player 2 is at (1, 0)
    
    """

    def __init__(self):
        self._board = [[0] * ylim for _ in range(xlim)]
        self._board[-1][-1] = 1  # block lower-right corner
        self._parity = 0
        self._player_locations = [None, None]

    def forecast_move(self, move):
        """ Return a new board object with the specified move
        applied to the current game state.
        
        Parameters
        ----------
        move: tuple
            The target position for the active player's next move
        """
        if move not in self.get_legal_moves():
            raise RuntimeError("Attempted forecast of illegal move")
        newBoard = deepcopy(self)
        newBoard._board[move[0]][move[1]] = 1
        newBoard._player_locations[self._parity] = move
        newBoard._parity ^= 1
        return newBoard

    def get_legal_moves(self):
        """ Return a list of all legal moves available to the
        active player.  Each player should get a list of all
        empty spaces on the board on their first move, and
        otherwise they should get a list of all open spaces
        in a straight line along any row, column or diagonal
        from their current position. (Players CANNOT move
        through obstacles or blocked squares.)
        """
        loc = self._player_locations[self._parity]
        if not loc:
            return self._get_blank_spaces()
        moves = []
        rays = [(1, 0), (1, -1), (0, -1), (-1, -1),
                (-1, 0), (-1, 1), (0, 1), (1, 1)]
        for dx, dy in rays:
            _x, _y = loc
            while 0 <= _x + dx < xlim and 0 <= _y + dy < ylim:
                _x, _y = _x + dx, _y + dy
                if self._board[_x][_y]:
                    break
                moves.append((_x, _y))
        return moves

    def _get_blank_spaces(self):
        """ Return a list of blank spaces on the board."""
        return [(x, y) for y in range(ylim) for x in range(xlim)
                if self._board[x][y] == 0]


### Part 3: Implementing the Minimax Algorithm